In [ ]:
# This notebook is for creating and testing method on curating datasets on stock trading data for offline reinforcement learning with decision transformer model
# This will first create a custom dataset class that can provide sliding window data
# Then it will use those datasets to create gym environments and sample state, action, reward which then store as a replay buffer
# Group these replay buffers and export as a dataset

In [1]:
# import helper function for getting stock data
from getstock import get_stock_data_yf_between_with_indicators

# get stock data with technical indicators
stock_name = 'AAPL'
start_date = '2019-01-01'
end_date = '2021-12-31'
interval = '1d'
indicators = ['volume_obv', 'trend_macd', 'momentum_rsi']

stockdata = get_stock_data_yf_between_with_indicators(stock_name, start_date, end_date, interval, indicators)


[*********************100%***********************]  1 of 1 completed


c:\Users\Victor\anaconda3\envs\pythonenv1\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\Victor\anaconda3\envs\pythonenv1\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [3]:
# create a custom dataset class that can provide sliding window data
import torch
from torch.utils.data import Dataset

class SlidingDataset(Dataset):
    def __init__(self, data, window_size, stride=1, transform=None):
        self.data = data
        self.window_size = window_size
        self.stride = stride
        self.transform = transform

    def __len__(self):
        return (len(self.data) - self.window_size) // self.stride + 1

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        start = idx * self.stride
        end = start + self.window_size
        sample = self.data[start:end]

        if self.transform:
            sample = self.transform(sample)

        return sample

OSError: [WinError 182] The operating system cannot run %1. Error loading "c:\Users\Victor\anaconda3\envs\pythonenv1\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
# create a sliding dataset and its dataloader
window_size = 30
stride = 5
dataset = SlidingDataset(stockdata, window_size, stride)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
# create a gym environment for each sliding window and sample state, action, reward